# Tools, Hallucinations, and ReAct Agents

## 1. What are *Tools* in LLMs?

**Tools** are external functions or systems that an LLM can call to get *real* information or perform *real* actions.

Instead of the model guessing, it can:
- Query a database
- Do math
- Call an API
- Search documents
- Execute code

Think of tools as **giving hands and eyes to a brain that can only talk**.

### Example (conceptual)
Without tools:
> “What is 78243 × 912?”  
LLM guesses → wrong answer.

With tools:
- LLM calls a calculator
- Returns the exact result

### Common tools
- Calculator
- Web search
- Vector database retriever
- SQL / NoSQL database
- Code execution environment
- Custom APIs

---

## 2. What Problem Do Tools Solve?

### Problem 1: LLMs are not grounded in reality
LLMs generate text based on patterns, not facts.

Tools let the model:
- Fetch **up-to-date data**
- Access **private or domain-specific info**
- Produce **verifiable outputs**

---

### Problem 2: LLMs are bad at precision
LLMs struggle with:
- Long math
- Exact counts
- Complex logic
- Date calculations

Tools handle precision; the LLM handles reasoning.

---

### Problem 3: Hallucinations
When an LLM doesn’t know something, it often *makes something up*.

Tools reduce hallucinations by forcing:
- Lookups instead of guesses
- Evidence-based responses
- Explicit failure when data is missing

---

## 3. What Are Hallucinations in LLMs?

A **hallucination** is when an LLM confidently produces **false or made-up information**.

### Why hallucinations happen
- LLMs predict *likely text*, not *true text*
- No internal fact-checking
- Training data is incomplete or outdated
- Model is pressured to always respond

### Example
> “Give me a research paper by Einstein on neural networks.”

LLM hallucination:
> “Einstein published ‘Neural Computation in 1912’…”

Reality:
❌ No such paper exists.

---

### Types of hallucinations
1. **Factual hallucination** – wrong facts
2. **Source hallucination** – fake citations or links
3. **Logical hallucination** – reasoning errors
4. **Tool hallucination** – inventing tool results

---

## 4. How Tools Reduce Hallucinations (But Don’t Eliminate Them)

Tools help by:
- Grounding answers in external data
- Limiting free-form guessing
- Providing verifiable outputs

But:
- The model can still misinterpret tool results
- Bad prompts = bad tool usage
- Tools only help if the model is *forced* to use them

Tools reduce hallucinations — they don’t magically remove them.

---

## 5. What Is a ReAct Agent?

**ReAct = Reason + Act**

A ReAct agent alternates between:
1. **Reasoning** (thinking about what to do)
2. **Acting** (calling a tool)
3. **Observing** (reading tool output)
4. Repeating until done

It’s a loop, not a single prompt.

---

### ReAct Pattern (conceptual)

```text
Question
 ↓
Thought: I need external info
 ↓
Action: call search tool
 ↓
Observation: search results
 ↓
Thought: now I can answer
 ↓
Final Answer


In [2]:
import os
import dotenv
dotenv.load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
from langchain_groq import ChatGroq


In [3]:

llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

In [4]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke('write an essay on llms')
ai_msg

AIMessage(content='**The Rise and Impact of Large Language Models (LLMs)**  \n\nIn the rapidly evolving landscape of artificial intelligence (AI), Large Language Models (LLMs) have emerged as transformative tools, reshaping how humans interact with technology. These models, capable of generating human-like text, answering complex questions, and performing a multitude of language-based tasks, are revolutionizing industries and challenging traditional notions of communication and problem-solving. From powering chatbots to aiding scientific research, LLMs are at the forefront of AI innovation. This essay explores the development, applications, benefits, and challenges of LLMs, while reflecting on their societal implications and future trajectory.  \n\n**Understanding LLMs: Training and Functionality**  \nLLMs are built on advanced neural network architectures, particularly the transformer model, which enables them to process and generate text efficiently. Trained on vast datasets comprisi

In [5]:
def decor(func):
    val = func()
    return val + " Decorated"

@decor
def get_me():
    return "Hello World"


print(get_me)#when get_me it calls decor(get_me) and returns val + " Decorated"


Hello World Decorated


In [6]:
def decor(func):
    def wrapper():
        val = func()
        return val + " Decorated"
    return wrapper


@decor
def get_me():
    return "Hello World"

print(get_me())#when get_me() it calls decor(get_me) and returns wrapper() and that means it calls wrapper() and returns val + " Decorated"


Hello World Decorated


In [7]:
def process(data):
    """
    Process input data and return cleaned output.

    Args:
        data (list): Raw input values

    Returns:
        list: Cleaned values
    """
    ...

help(process)

Help on function process in module __main__:

process(data)
    Process input data and return cleaned output.

    Args:
        data (list): Raw input values

    Returns:
        list: Cleaned values



In [8]:
# Tools
#lets see what tools are and how they work and used in llm to get useful output for cases where llms alone can't get useful output
# we will use langchain tool decorator and feed the function to the tool and let llm call the tool in case it needs to

from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather in a given location"""
    return f"The weather in {location} is sunny"

# now we feed this get_weather function to the tool and let llm call the tool in case it needs to

model = llm.bind_tools([get_weather]) # here we give list of tools to the model

msg = model.invoke("What is the weather in New York?")#now it does not try to answer from data that the model is not trained on instead it search for tools

# we can also use tool for other purposes like calling APIs, databases, etc thats why it allows us to make agents using tool

print(msg)



content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in New York. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. The required parameter is location, which should be a string. Since the user specified New York, I need to call get_weather with location set to "New York". I\'ll make sure the JSON is correctly formatted with the name of the function and the arguments as specified.\n', 'tool_calls': [{'id': 'jh4vhy6nv', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 155, 'total_tokens': 267, 'completion_time': 0.154009134, 'completion_tokens_details': {'reasoning_tokens': 87}, 'prompt_time': 0.006168125, 'prompt_tokens_details': None, 'queue_time': 0.051201375, 'total_time': 0.160177259}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'servic

In [9]:
print(msg.additional_kwargs['reasoning_content'])

Okay, the user is asking for the weather in New York. Let me check the tools available. There's a function called get_weather that takes a location parameter. The required parameter is location, which should be a string. Since the user specified New York, I need to call get_weather with location set to "New York". I'll make sure the JSON is correctly formatted with the name of the function and the arguments as specified.



In [10]:
#now we need tool call loop so that we can call the tool then return the result to the LLM and llm can use it to give the final answer

print(msg.tool_calls)


[{'name': 'get_weather', 'args': {'location': 'New York'}, 'id': 'jh4vhy6nv', 'type': 'tool_call'}]


In [15]:
#model generates tool calls

msg = [{
    'role': 'user',
    'content': 'What is the weather in Kathmandu?'
}]

model = model.bind_tools([get_weather])
msg_ai = model.invoke(msg)
print(msg_ai)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Kathmandu. I need to use the get_weather function. The function requires a location parameter. Kathmandu is the location here. So I should call get_weather with location set to "Kathmandu". Let me make sure there\'s no typo. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': 'sjynqhv62', 'function': {'arguments': '{"location":"Kathmandu"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 156, 'total_tokens': 259, 'completion_time': 0.158365477, 'completion_tokens_details': {'reasoning_tokens': 76}, 'prompt_time': 0.00622774, 'prompt_tokens_details': None, 'queue_time': 0.158972714, 'total_time': 0.164593217}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq

In [16]:
msg.append(msg_ai)
msg

[{'role': 'user', 'content': 'What is the weather in Kathmandu?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Kathmandu. I need to use the get_weather function. The function requires a location parameter. Kathmandu is the location here. So I should call get_weather with location set to "Kathmandu". Let me make sure there\'s no typo. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': 'sjynqhv62', 'function': {'arguments': '{"location":"Kathmandu"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 156, 'total_tokens': 259, 'completion_time': 0.158365477, 'completion_tokens_details': {'reasoning_tokens': 76}, 'prompt_time': 0.00622774, 'prompt_tokens_details': None, 'queue_time': 0.158972714, 'total_time': 0.164593217}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_d

In [ ]:
# now we will call the tool calls
for tool_call in msg_ai.tool_calls:
    result = get_weather.invoke(tool_call)#execute tool 
    msg.append(result)

# call model with result

message_result_after_context = model.invoke(msg)#provide context to llm now
print(message_result_after_context)




content='The weather in Kathmandu is currently sunny. A perfect day to enjoy outdoor activities! 😊' additional_kwargs={'reasoning_content': "Okay, the user asked about the weather in Kathmandu. I called the get_weather function with location set to Kathmandu. The response came back saying it's sunny. Now I need to present this information clearly. Let me check if there's any additional details I should mention. The function only provided the weather condition, so I'll stick to that. Maybe add a friendly note about enjoying the weather. Keep it concise and straightforward.\n"} response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 213, 'total_tokens': 327, 'completion_time': 0.202650978, 'completion_tokens_details': {'reasoning_tokens': 89}, 'prompt_time': 0.013415462, 'prompt_tokens_details': None, 'queue_time': 0.058700748, 'total_time': 0.21606644}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reaso

In [19]:
print(model)

bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001B5FC2D7770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B5FC4BC2F0>, model_name='qwen/qwen3-32b', temperature=1e-08, reasoning_format='parsed', model_kwargs={}, groq_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather in a given location', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]} config={} config_factories=[]
